<a href="https://colab.research.google.com/github/CODERZZZ123/StockPred_NewsHeadlines/blob/master/pred_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
import nltk.corpus
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import euclidean_distances
!pip install textattack

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  Using cached textattack-0.3.8-py3-none-any.whl (418 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached flair-0.12.2-py3-none-any.whl (373 kB)
  Using cached language_tool_python-2.7.1-py3-none-any.whl (34 kB)
  Using cached lemminflect-0.2.3-py3-none-any.whl (769 kB)
  Using cached lru_dict-1.2.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (28 kB)
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached word2number-1.1.zip (9.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached num2words-0.5.12-py3-none-any.whl (125 kB)
  Using cached pinyin-0.4.0.tar.gz (3.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached OpenHowNet-2.0-py3-none-any.whl (18 kB)
  Using cached pycld2-0.41.tar.gz (41.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached cl

In [26]:
from google.colab import drive
drive.mount('/content/drive')
# import requests
# import time

# while True:
#     try:
#         requests.get('https://www.google.com')
#         print("Kept alive.")
#     except:
#         print("Failed to keep alive.")
#     time.sleep(600)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
from sklearn import  set_config
set_config(display = 'diagram')

In [28]:
df_NewsDjia = pd.read_csv('News_DJIA.csv', delimiter=',')
df_RedditNews = pd.read_csv('News_reddit.csv', delimiter=',')
df_DjiaStock = pd.read_csv('DJIA_t.csv',delimiter=',')

In [29]:
for i in df_NewsDjia.columns :
    df_NewsDjia[i] = df_NewsDjia[i].astype(str)

In [30]:
df_NewsDjia

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [31]:
num_columns = df_NewsDjia.columns.shape[0]
col_names = df_NewsDjia.columns.values.tolist()
col_names = col_names[1:]
df_NewsDjia.loc[:, 'merged'] = " "
for each_col_ind in range(num_columns-1):
    df_NewsDjia.loc[:, 'merged'] =  df_NewsDjia.loc[:, 'merged'] + " " + df_NewsDjia[col_names[each_col_ind]] + " "

In [32]:
df_finalised = df_NewsDjia[['merged']]

In [33]:
date_numric_stopwords = "THOUSAND MILLION BILLION TRILLION HUNDRED ANNUAL ANNUALLY ANNUM YEAR YEARLY QUARTER QUARTERLY QTR MONTH MONTHLY WEEK WEEKLY DAY DAILY JANUARY FEBRUARY MARCH APRIL MAY JUNE JULY AUGUST SEPTEMBER OCTOBER NOVEMBER DECEMBER JAN FEB MAR APR MAY JUN JUL AUG SEP SEPT OCT NOV DEC MONDAY TUESDAY WEDNESDAY THURSDAY FRIDAY SATURDAY SUNDAY TWO THREE FOUR FIVE SIX SEVEN EIGHT NINE TEN ELEVEN TWELVE THIRTEEN FOURTEEN FIFTEEN SIXTEEN SEVENTEEN EIGHTEEN NINETEEN TWENTY THIRTY FORTY FIFTY SIXTY SEVENTY EIGHTY NINETY FIRST SECOND THIRD FOURTH FIFTH SIXTH SEVENTH EIGHTH NINTH TENTH II III IV V VI VII VIII IX X XI XII XIII XIV XV XVI XVII XVIII XIX XX"
date_numric_stopwords = date_numric_stopwords.lower()
date_numric_stopwords = date_numric_stopwords.split(" ")

In [34]:

def remove_unecssary_word(sent):
    sent = sent.replace("b'",'')
    sent = sent.replace('b"','')
    sent = sent.lower()
    sent = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sent)
    stop = stopwords.words('english')
    stop.extend(date_numric_stopwords)
    sent = " ".join([word for word in sent.split() if word not in (stop)])
    sent = " ".join([WordNetLemmatizer().lemmatize(word) for word in sent.split() ])
    return sent

In [35]:
df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)

<ipython-input-35-6847205c14ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)


In [36]:
df_finalised.iloc[0][0]

'0 georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick old raped nothing 150 russian tank entered south ossetia whilst georgia shoot russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war start today georgia invades s

In [37]:
# from textattack.augmentation import EmbeddingAugmenter
# embed_aug = EmbeddingAugmenter()
# embed_aug.augment(df_finalised.iloc[0][0])

In [38]:
# from textattack.augmentation import WordNetAugmenter
# wordnet_aug = WordNetAugmenter()
# wordnet_aug.augment(df_finalised.iloc[0][0])

In [39]:
# from textattack.augmentation import EasyDataAugmenter
# eda_aug = EasyDataAugmenter()
# eda_aug.augment(df_finalised.iloc[0][0])

In [40]:
df_finalised = pd.concat([df_finalised,df_NewsDjia['Label']],axis = 1)

In [41]:
Positive_word = ""
with open('Positive_words.txt', 'r') as file:
    Positive_word = file.read().replace('\n', '')
Positive_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Positive_word.split(',') ])

In [42]:
Negative_word = ""
with open('Negative_words.txt','r') as file:
    Negative_word = file.read().replace('\n','')
Negative_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Negative_word.split(',') ])

In [43]:
Positive_word = Positive_word.split(',')
Negative_word = Negative_word.split(',')
Negative_word = Negative_word[0].split()
Positive_word = Positive_word[0].split()

In [44]:
set_post = set(Positive_word)
set_neg = set(Negative_word)
def Count(obj):
    count_post = 1
    count_neg = 1
    obj = obj.split()
    for i in obj:
        if i in set_post:
            count_post = count_post + 1
        elif i in set_neg:
            count_neg = count_neg + 1
    return count_post/count_neg


In [45]:
df_finalised['ratio_pos_neg'] = df_finalised['merged'].apply(Count)
X_data = df_finalised['merged']
# ratio_column = df_finalised['ratio_pos_neg']
# ratio_column = np.array(ratio_column).reshape((-1,1))

In [46]:
Y_data = df_finalised['Label']
Y_data = Y_data.astype(int)

In [ ]:
from textattack.augmentation import EasyDataAugmenter
eda_aug = EasyDataAugmenter()
data_X = []
label = []
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'w').close()
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','w').close()
with open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'a') as f1 , open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','a') as f2:
  for i in range(1820,X_data.shape[0]):
    data_X.append(X_data[i])
    label.append(Y_data[i])
    f1.write(X_data[i])
    f1.write(",")
    y = str(Y_data[i])
    f2.write(y)
    f2.write(",")
    if i%10 == 0:
      data = eda_aug.augment(X_data[i])
      for d in data:
        data_X.append(d)
        label.append(Y_data[i])
        f1.write(d)
        f1.write(",")
        f2.write(y)
        f2.write(",")

    print(i)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879


In [ ]:
Y_data = np.array(Y_data)
Y_data

In [ ]:
X_data[0]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data_X

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
X_data.shape

In [ ]:
x_train , x_test , y_train, y_test = X_data[:1590] , X_data[1591:] , Y_data[:1590] , Y_data[1591:]

In [ ]:
tf_idf = TfidfVectorizer(min_df=10,ngrams = (2,2))
x_train_tf = tf_idf.fit_transform(x_train)
x_test_tf = tf_idf.transform(x_test)
x_train = x_train_tf.toarray()
x_test = x_test_tf.toarray()

In [ ]:
mean_not_zero = x_train[x_train != 0].mean()
x_train[x_train!=0] = x_train[x_train!=0] -  mean_not_zero
x_test[x_test != 0 ] = x_test[x_test != 0] - mean_not_zero

In [ ]:
# ct_vect = CountVectorizer()
# x_train = ct_vect.fit_transform(x_train).toarray()
# x_test = ct_vect.transform(x_test).toarray()

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# pca = PCA()
# pca.fit(x_train_stand)
# cumsum = np.cumsum(pca.explained_variance_ratio_)*100
# d = [n for n in range(len(cumsum))]
# plt.figure(figsize=(10, 10))
# plt.plot(d,cumsum, color = 'red',label='cumulative explained variance')
# plt.title('Cumulative Explained Variance as a Function of the Number of Components')
# plt.ylabel('Cumulative Explained variance')
# plt.xlabel('Principal components')
# plt.axhline(y = 95, color='k', linestyle='--', label = '95% Explained Variance')
# plt.legend(loc='best')

In [ ]:
# VARIANCE EXPLAINED 90%
pca = PCA(0.90)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)
pca.explained_variance_ratio_

In [ ]:
# K = [int(i) for i in range(x_train.shape[0]//3)]
# param_grid = {
#     'n_neighbors' : K
# }
# knn = KNeighborsClassifier()
# grid_search = GridSearchCV(estimator=knn,scoring='accuracy',param_grid=param_grid,cv = 10,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# grid_search.best_estimator_
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(x_train,y_train)
# y_pred = knn.predict(x_test)
# print(accuracy_score(y_pred,y_test))
# print(classification_report(y_pred,y_test))

In [ ]:
# C = [0.01 , 0.05  , 0.08, 0.1, 0.3 , 0.6 , 0.9 , 1.2 , 1.7 , 2.5 , 2.9 , 3.7, 4.6 , 5.1 , 5.9 , 6.9 , 7.5 , 8.3 , 9.3 , 10]
# kernel = ['linear', 'poly', 'rbf', 'sigmoid']
# gamma = ['scale','auto']
# degree = [int(x) for x in range(1,5)]
# param_grid = {
#     'kernel' : kernel, vg
#     'gamma' : gamma,
#     'degree' : degree,
#     'C' : C
# }
# svc = SVC()
# grid_search = GridSearchCV(estimator=svc,scoring='accuracy',param_grid=param_grid,cv = 5,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# best_gird_search = grid_search.best_estimator_
# from sklearn.metrics import accuracy_score
# y_pred = best_gird_search.predict(x_test)
# print(confusion_matrix(y_test,y_pred))
# print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
# print(accuracy_score(y_train,best_gird_search.predict(x_train)))
# Score = cross_val_score(best_gird_search,X_data,Y_data,cv = 5)
# Score

In [ ]:
from sklearn.metrics import accuracy_score
model = SVC(C = 25, degree= 4, gamma =  'auto', kernel = 'linear')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy for the trained data is {}".format(accuracy_score(y_train,model.predict(x_train))))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("classification report {}".format(classification_report(y_test,y_pred)))
# Score = cross_val_score(model,X_data,Y_data,cv = 5)
# Score

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# model = Sequential()
# model.add(Dense(12,input_shape = (x_train.shape[1],),activation = 'relu'))
# model.add(Dense(8,activation = 'relu'))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train,y_train,epochs=200,batch_size=1000)
# predictions = (model.predict(x_test) > 0.5).astype(int)
# abs(predictions.T - y_test).sum()/y_test.shape[0]


In [ ]:
# randomclassifier=RandomForestClassifier(n_estimators=1000,criterion='entropy')
# randomclassifier.fit(x_train,y_train)
# predictions = randomclassifier.predict(x_test)
# matrix=confusion_matrix(y_test,predictions)
# print(matrix)
# score=accuracy_score(y_test,predictions)
# print(score)
# report=classification_report(y_test,predictions)
# print(report)